## Importation des fichiers utiles

In [1]:
import urllib.request
import json
import pandas as pd
import numpy as np
from PIL import Image, ExifTags
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.linear_model import Perceptron

NOMBRE_IMAGE=141

<br>

## Demande de Json
Nous Faisons une Requète WikiData pour demander un Json contenant le lien menant à des libres d'une certaine catégorie. Nous avons essayé plusieurs types d'images différentes :

mythical creature = Q2239243<br>
photograph = Q125191<br>
flower = Q506<br>
fruit = Q1364<br>
Chien = Q144<br>
lac  = Q23397<br>


SELECT ?image ?im WHERE {
  ?image wdt:P31 wd:Q506.
  ?image wdt:P18 ?im
  }
LIMIT 200

In [36]:
lstSources = [1364,144,23397]
limitPerSources = 5
sources = {}
for cat in lstSources:
    url ="https://query.wikidata.org/sparql?query=SELECT%20%3Fimage%20%3Fim%20WHERE%20%7B%0A%20%20%3Fimage%20wdt%3AP31%20wd%3AQ"+str(cat)+".%0A%20%20%3Fimage%20wdt%3AP18%20%3Fim%0A%20%20%7D%0ALIMIT%20"+str(limitPerSources)+"&format=json"
    response = urllib.request.urlopen(url)
    myJson = json.loads(response.read().decode("utf-8"))
    if cat == lstSources[0]:
        sources = myJson
    else:
        sources["results"]["bindings"] = sources["results"]["bindings"]+myJson["results"]["bindings"]
myJsonNormalized = pd.json_normalize(sources)

<br>

## Télechargement des images ##
A partir du Json contenant les liens menant aux images, nous lancons des requètes URLLIB pour télecharger ces images.

In [28]:
#telechargement des images

responseJson = myJsonNormalized["results.bindings"]
index=0
for i in range(len(responseJson[0])):   
    imageLink = responseJson[0][i]["im"]["value"]
    print(index)
    
    im = "./images/image"+str(index)+".jpg"
    urllib.request.urlretrieve(imageLink, im)
    try:
        imgfile = Image.open(im).convert("RGB")
    except:
        index-=1
        print("pas fonctionne")
    index +=1
NOMBRE_IMAGE=index+1


14

<br>

## Récupération des métadonnées
L'obhectif de cette partie est de récupérer les informations des images, nous récupérons les images suivantes :<br>
1. Récupération Exif utiles
2. Récupération couleurs dominantes (K-means)
3. Récupération taille des images
4. Détermination portrait ou paysage

Apres cela, nous avons creer un Json **metaDataJson.json** contenant toutes les informationns precedentes

In [66]:
# Récupération des métadonnées

metaData ={}
for i in range(10):
    im = "./images/image"+str(i)+".jpg"
    try:
        imgfile = Image.open(im).convert("RGB")

        #Recupération Exif utiles
        exif = {}
        exifCritere= ("Model","Make","ImageDescription","ExposureTime","GPSInfo","ISO","DateTimeOriginal")
        for k,v in  imgfile.getexif().items() :
            if k in ExifTags.TAGS :
                if ExifTags.TAGS[k] in exifCritere:
                    exif = {ExifTags.TAGS[k]:v}

        # Recupération couleurs dominantes (K-means)
        numarray = np.array(imgfile.getdata(), np.uint8)
        clusters  = KMeans(n_clusters=3)
        clusters.fit(numarray[::50])
        L = clusters.cluster_centers_.astype(int).tolist()

        # Recupération taille des images
        width = imgfile.width
        height = imgfile.height
        mode = imgfile.mode

        # Détermination portrait ou paysage
        if abs(1-width/height)<0.1 :
            orientation='carre' 
        elif width>height:
            orientation='paysage'
        else: 
            orientation="portrait"

        metaData["image"+str(i)] = { "width" : width, "height": height, "exif": exif, "mode":mode, "tags":[], "couleurs":L, "orientation":orientation }
    except Exception:
        print("l'image "+str(i) +" ne peut pas etre ouverte")
metaData

metaDatajson = json.dumps(metaData)
f = open("metaDataJson.json", "w")
f.write(metaDatajson)
f.close()



## Analyse

In [4]:
def analyse(nomUser):
      jsonMeta=json.load(open("metaDatajson.json"))
      jsonMetaNormalized = pd.json_normalize(jsonMeta)
    

      jsonUser=json.load(open("jsonUser.json"))
      jsonUserNormalized = pd.json_normalize(jsonUser)
      
      
      
      L=[]
      for i,obj in jsonUserNormalized.iterrows():
            L.append(obj["data.like"])
      
      """ for nameImage in L:
            numarray.add(jsonMetaNormalized["image1"]["couleurs"])    
      numarray = np.array(jsonMetaNormalized["image1"]["couleurs"])
      result = np.array(jsonUserNormalized["image1"]["couleurs"])

      
      perceptron = Perceptron(max_iter=1000)
      perceptron.fit(numarray, result)
      
      x_predict = np.array( [jsonMetaNormalized["image1"]["couleurs"],jsonMetaNormalized["image2"]["couleurs"] ,jsonMetaNormalized["image6"]["couleurs"] , jsonMetaNormalized["image16"]["couleurs"]])

      y_predict = perceptron.predict(x_predict)
      print(y_predict) """
analyse()

[['image1', 'image10', 'image27', 'image45'], ['image2', 'image17', 'image37', 'image69'], ['image6', 'image12', 'image78', 'image100'], ['image16', 'image25', 'image45', 'image52']]


## Brouillon


In [39]:
""" def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

def analyse(numUser):
    jsonMeta=json.load(open("metaDatajson.json"))
    jsonMetaNormalized = pd.json_normalize(jsonMeta)
    

    jsonUser=json.load(open("jsonUser.json"))
    jsonUserNormalized = pd.json_normalize(jsonUser)
    
    #L = jsonUserNormalized[numUser+1]["data"]["like"]
    L=[]
    for i,obj in jsonUserNormalized.iterrows():
        L.append(obj["data.like"][numUser])
    
    c=np.array([[0,0,0],[0,0,0],[0,0,0]])
    
    orientationPref=[]
    for nameImage in L:
        for j in range(len(c)):
            print(nameImage)
            print(jsonMetaNormalized[nameImage])
            c[0]+=np.square(np.array(jsonMetaNormalized[nameImage]['couleurs'][0][j]))
            c[1]+=np.square(np.array(jsonMetaNormalized[nameImage]['couleurs'][1][j]))
            c[2]+=np.square(np.array(jsonMetaNormalized[nameImage]['couleurs'][2][j]))

        #orientationPref.append(jsonUserNormalized[image]['exif']['model'])

    newColor=[]
    for i in range(len(c)):
        newColor.append(np.sqrt((c[i])/len(c)),np.sqrt(c[i]/len(c)),np.sqrt((c[i])/len(c)))
    #newOrientation = most_frequent(orientationPref)
    newOrientation = 0
    return (newColor,newOrientation)
a=analyse(0)
print(a[0]) """

image1


KeyError: 'image1'

In [56]:
# def visualisation(parametres,nameUser='ALL'):
    
#     myJson = json.loads("metaDataJson.json")
#     myJsonNormalized = pd.json_normalize(myJson)
#     L=[]

#     jsonUser=json.loads("jsonUser.json")
#     jsonUserNormalized = pd.json_normalize(jsonUser)
#     L = jsonUserNormalized[nameUser]["image"]
#     yearData =[]
#     modelData =[]
#     for image in L:
#         if 'year' in myJsonNormalized[image]['exif']:
#             yearData.append(myJsonNormalized[image]['exif']['year'])
#         if 'mode1' in myJsonNormalized[image]:
#             modelData.append(myJsonNormalized[image]['exif']['model'])
#     max(yearData, key=yearData.count)
#     max(modelData, key=modelData.count)

# visualisation